In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [2]:
from rec_to_binaries.read_binaries import readTrodesExtractedDataFile

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

readTrodesExtractedDataFile(
    '../test_data/lotus/preprocessing/20190902/20190902_lotus_06_r3.time/20190902_lotus_06_r3.continuoustime.dat')

/home/edeno/Documents/Github/rec_to_binaries/rec_to_binaries/read_binaries.py:70: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(typearr)


{'description': 'Continuous time periods (start and end times)',
 'byte_order': 'little endian',
 'original_file': '20190902_lotus_06_r3.rec',
 'clockrate': '30000',
 'system_time_at_creation': '1567459146943',
 'timestamp_at_creation': '390829088',
 'trodes_version': '1.8.2',
 'compile_date': 'May 16 2019',
 'compile_time': '10:32:19',
 'qt_version': '5.9.8',
 'commit_tag': 'heads/Release_1.8.2-0-g9a3e37c',
 'controller_firmware': '0.0',
 'headstage_firmware': '3.9',
 'autosettle': '0',
 'smartref': '0',
 'gyro': '0',
 'accelerometer': '0',
 'magnetometer': '0',
 'time_offset': '0',
 'fields': '<trodestime uint32><systime int64>',
 'data': array([(390923532, 1567459150091663430), (390923533, 1567459150091671120),
        (390923534, 1567459150091674111), ...,
        (443082985, 1567460888733625293), (443082986, 1567460888733629636),
        (443082987, 1567460888733631849)],
       dtype=[('trodestime', '<u4'), ('systime', '<i8')])}

In [73]:
from rec_to_binaries.adjust_timestamps import _label_time_chunks, _regress_timestamps

continuoustime_filename = '../test_data/lotus/preprocessing/20190902/20190902_lotus_06_r3.time/20190902_lotus_06_r3.continuoustime.dat'

data_file = readTrodesExtractedDataFile(continuoustime_filename)
new_data = (
    pd.DataFrame(data_file['data'])
    .assign(
        time_chunk_label=lambda df: _label_time_chunks(df.trodestime))
    .assign(
        adjusted_systime=lambda df: _regress_timestamps(df.trodestime,
                                               df.systime)))
new_data

/home/edeno/Documents/Github/rec_to_binaries/rec_to_binaries/read_binaries.py:70: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(typearr)


,trodestime,systime,time_chunk_label,adjusted_systime
0,390923532,1567459150091663430,0,1567459150091796480
1,390923533,1567459150091671120,0,1567459150091829504
2,390923534,1567459150091674111,0,1567459150091863040
3,390923535,1567459150091676962,0,1567459150091896320
4,390923536,1567459150091679726,0,1567459150091929600
...,...,...,...,...
52159451,443082983,1567460888733526586,0,1567460888733756416
52159452,443082984,1567460888733528880,0,1567460888733789952
52159453,443082985,1567460888733625293,0,1567460888733822976
52159454,443082986,1567460888733629636,0,1567460888733856512


In [72]:
from rec_to_binaries.create_system_time import infer_systime

new_data2 = infer_systime(data_file)
new_data2

,trodestime,systime,time_chunk_label,adjusted_systime
0,390923532,1567459146943000000,1,1567459146943000000
1,390923533,1567459146943033333,1,1567459146943033333
2,390923534,1567459146943066666,1,1567459146943066666
3,390923535,1567459146943100000,1,1567459146943100000
4,390923536,1567459146943133333,1,1567459146943133333
...,...,...,...,...
52159451,443082983,1567460885591366666,1,1567460885591366666
52159452,443082984,1567460885591400000,1,1567460885591400000
52159453,443082985,1567460885591433333,1,1567460885591433333
52159454,443082986,1567460885591466666,1,1567460885591466666


In [74]:
new_data.dtypes, new_data2.dtypes

(trodestime          uint32
 systime              int64
 time_chunk_label     int64
 adjusted_systime     int64
 dtype: object,
 trodestime          uint32
 systime              int64
 time_chunk_label     int64
 adjusted_systime     int64
 dtype: object)

In [75]:
np.diff(new_data.adjusted_systime)

array([33024, 33536, 33280, ..., 33024, 33536, 33280])

In [76]:
np.diff(new_data2.adjusted_systime)

array([33333, 33333, 33334, ..., 33333, 33333, 33334])

In [77]:
new_data.adjusted_systime[0], new_data.systime[0]

(1567459150091796480, 1567459150091663430)

In [78]:
new_data2.adjusted_systime[0]

1567459146943000000

In [79]:
pd.to_datetime(new_data2.adjusted_systime[0])

Timestamp('2019-09-02 21:19:06.943000')

In [80]:
pd.to_datetime(new_data.adjusted_systime[0])

Timestamp('2019-09-02 21:19:10.091796480')

In [64]:
pd.to_datetime(new_data.systime[0])

Timestamp('2019-09-02 21:19:10.091663430')

In [67]:
pd.to_datetime(int(data_file['system_time_at_creation']), unit='ms')

Timestamp('2019-09-02 21:19:06.943000')

In [69]:
t = pd.to_datetime(int(data_file['system_time_at_creation']), unit='ms')
t

Timestamp('2019-09-02 21:19:06.943000')

In [71]:
np.log(t.value)

41.89598760452183

In [36]:
pd.to_datetime

Signature:
pd.to_datetime(
    arg,
    errors='raise',
    dayfirst=False,
    yearfirst=False,
    utc=None,
    format=None,
    exact=True,
    unit=None,
    infer_datetime_format=False,
    origin='unix',
    cache=True,
)
Docstring:
Convert argument to datetime.

Parameters
----------
arg : int, float, str, datetime, list, tuple, 1-d array, Series DataFrame/dict-like
    The object to convert to a datetime.
errors : {'ignore', 'raise', 'coerce'}, default 'raise'
    - If 'raise', then invalid parsing will raise an exception.
    - If 'coerce', then invalid parsing will be set as NaT.
    - If 'ignore', then invalid parsing will return the input.
dayfirst : bool, default False
    Specify a date parse order if `arg` is str or its list-likes.
    If True, parses dates with the day first, eg 10/11/12 is parsed as
    2012-11-10.
    with day first (this is a known bug, based on dateutil behavior).
yearfirst : bool, default False
    Specify a date parse order if `arg` is str or its

In [29]:
data_file['system_time_at_creation']

'1567459146943'

In [30]:
data_file

{'description': 'Continuous time periods (start and end times)',
 'byte_order': 'little endian',
 'original_file': '20190902_lotus_06_r3.rec',
 'clockrate': '30000',
 'system_time_at_creation': '1567459146943',
 'timestamp_at_creation': '390829088',
 'trodes_version': '1.8.2',
 'compile_date': 'May 16 2019',
 'compile_time': '10:32:19',
 'qt_version': '5.9.8',
 'commit_tag': 'heads/Release_1.8.2-0-g9a3e37c',
 'controller_firmware': '0.0',
 'headstage_firmware': '3.9',
 'autosettle': '0',
 'smartref': '0',
 'gyro': '0',
 'accelerometer': '0',
 'magnetometer': '0',
 'time_offset': '0',
 'fields': '<trodestime uint32><systime int64>',
 'data': array([(390923532, 1567459150091663430), (390923533, 1567459150091671120),
        (390923534, 1567459150091674111), ...,
        (443082985, 1567460888733625293), (443082986, 1567460888733629636),
        (443082987, 1567460888733631849)],
       dtype=[('trodestime', '<u4'), ('systime', '<i8')])}

In [34]:
from rec_to_binaries.adjust_timestamps import _insert_new_data

_insert_new_data(data_file, new_data)

{'description': 'Continuous time periods (start and end times)',
 'byte_order': 'little endian',
 'original_file': '20190902_lotus_06_r3.rec',
 'clockrate': '30000',
 'system_time_at_creation': '1567459146943',
 'timestamp_at_creation': '390829088',
 'trodes_version': '1.8.2',
 'compile_date': 'May 16 2019',
 'compile_time': '10:32:19',
 'qt_version': '5.9.8',
 'commit_tag': 'heads/Release_1.8.2-0-g9a3e37c',
 'controller_firmware': '0.0',
 'headstage_firmware': '3.9',
 'autosettle': '0',
 'smartref': '0',
 'gyro': '0',
 'accelerometer': '0',
 'magnetometer': '0',
 'time_offset': '0',
 'fields': '<trodestime uint32><systime int64><time_chunk_label int64><adjusted_systime int64>',
 'data': array([(390923532, 1567459150091663430, 0, 1567459150091796480),
        (390923533, 1567459150091671120, 0, 1567459150091829504),
        (390923534, 1567459150091674111, 0, 1567459150091863040), ...,
        (443082985, 1567460888733625293, 0, 1567460888733822976),
        (443082986, 156746088873362

In [35]:
from rec_to_binaries.adjust_timestamps import _insert_new_data

_insert_new_data(data_file, new_data2)

{'description': 'Continuous time periods (start and end times)',
 'byte_order': 'little endian',
 'original_file': '20190902_lotus_06_r3.rec',
 'clockrate': '30000',
 'system_time_at_creation': '1567459146943',
 'timestamp_at_creation': '390829088',
 'trodes_version': '1.8.2',
 'compile_date': 'May 16 2019',
 'compile_time': '10:32:19',
 'qt_version': '5.9.8',
 'commit_tag': 'heads/Release_1.8.2-0-g9a3e37c',
 'controller_firmware': '0.0',
 'headstage_firmware': '3.9',
 'autosettle': '0',
 'smartref': '0',
 'gyro': '0',
 'accelerometer': '0',
 'magnetometer': '0',
 'time_offset': '0',
 'fields': '<trodestime uint32><systime int64><time_chunk_label int64><adjusted_systime int64>',
 'data': array([(390923532, 1567459146943, 1, 1567459146943),
        (390923533, 1567459180276, 1, 1567459180276),
        (390923534, 1567459213609, 1, 1567459213609), ...,
        (443082985, 3306107580276, 1, 3306107580276),
        (443082986, 3306107613609, 1, 3306107613609),
        (443082987, 330610764

In [53]:
'systime' not in data_file['data'].dtype.names

False

8192